In [ ]:
import requests
from bs4 import BeautifulSoup
import pickle
import os
from tqdm import tqdm

root_url = "https://gameknot.com"

saved_links = []

os.makedirs("saved_files", exist_ok=True)

text_file_path = "saved_files/saved_links.txt"

for page_index in tqdm(range(290), desc="Processing pages"):
    page_url = f"https://gameknot.com/list_annotated.pl?u=all&c=0&sb=0&rm=0&rn=0&rx=9999&sr=0&p={page_index}"

    r = requests.get(page_url)
    
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        
        for elem in soup.find_all('tr', class_=["evn_list", "odd_list"]):
            list_of_links = elem.find_all('a')
            
            if len(list_of_links) > 1:
                href = list_of_links[1].get('href')
                if href:
                    full_link = root_url + href
                    saved_links.append(full_link)
        
        #print(f"Processed page {page_index}")
    else:
        #print(f"Failed to fetch page {page_index} with status code {r.status_code}")
        break

pickle_file_path = "saved_files/saved_links.p"
with open(pickle_file_path, "wb") as pickle_file:
    pickle.dump(saved_links, pickle_file)

with open(text_file_path, "w") as text_file:
    for link in saved_links:
        text_file.write(link + "\n")

print(f"Saved {len(saved_links)} links:")
print(f"- Pickle file: {pickle_file_path}")
print(f"- Text file: {text_file_path}")


Processing pages: 100%|██████████| 290/290 [04:16<00:00,  1.13it/s]

Saved 11600 links:
- Pickle file: saved_files/saved_links.p
- Text file: saved_files/saved_links.txt


In [18]:
import requests
from bs4 import BeautifulSoup
import os
import json
from tqdm import tqdm

links_file_path = "saved_files/saved_links.txt"

output_dir_path = "saved_files"
os.makedirs(output_dir_path, exist_ok=True)

output_file_path = os.path.join(output_dir_path, "commentary.txt")

all_data = []

with open(links_file_path, "r") as file:
    links = [line.strip() for line in file.readlines()]

with open(output_file_path,'w') as f:
    for idx, link in enumerate(tqdm(links[:10]), start=1):#Sample
        try:
            r = requests.get(link)
            
            if r.status_code == 200:
                soup = BeautifulSoup(r.content, 'html.parser')
                
                rows = soup.find_all('tr')
                
                for row in rows:
                    move_td = row.find('td', rowspan='2', style=lambda value: value and 'vertical-align: top' in value)
                    comment_td = row.find('td', style="vertical-align: top;")
                    
                    if move_td and comment_td:
                        move_text = move_td.get_text(separator=' ', strip=True)
                        move_text = ' '.join(move_text.split())
                        
                        comment_text = comment_td.get_text(separator=' ', strip=True)
                        comment_text = ' '.join(comment_text.split())

                        f.write(f"{move_text} <sep> {comment_text}\n")
                
            else:
                print(f"Failed to fetch link {idx} with status code {r.status_code}")
        except Exception as e:
            print(f"Error while processing link {idx}: {e}")

print(f"Scraped data for {len(all_data)} entries.")
print(f"All data saved in: {output_file_path}")

100%|██████████| 10/10 [00:07<00:00,  1.36it/s]

Scraped data for 0 entries.
All data saved in: saved_files/commentary.txt
